# Using MLflow models in batch deployments

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

In [ ]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Let's verify if the model we want to deploy, `heart-classifier`, is registered in the model registry. If not, we will register it from a local version we have in the repository:

In [ ]:
model_name = "heart-classifier"
model_local_path = "heart-classifier-mlflow/model"

In [ ]:
if not any(filter(lambda m: m.name == model_name, ml_client.models.list())):
    print(f"Model {model_name} is not registered. Creating...")
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
    )

Let's get the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

First, let's create the endpoint that is going to host the batch deployments. Remember that each endpoint can host multiple deployments at any time, however, only one of them is the default one:

In [ ]:
endpoint = BatchEndpoint(
    name="heart-classifier-batch",
    description="A heart condition classifier for batch inference",
)

In [ ]:
ml_client.batch_endpoints.begin_create_or_update(endpoint)

Batch endpoints can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [ ]:
compute_name = "cpu-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(
        name=compute_name, description="amlcompute", min_instances=0, max_instances=5
    )
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [ ]:
from time import sleep

print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

Let's create a deployment under the given endpoint.

In [ ]:
from azure.ai.ml.constants import BatchDeploymentOutputAction
from azure.ai.ml.entities import BatchRetrySettings

deployment = BatchDeployment(
    name="classifier-xgboost",
    description="A heart condition classifier based on XGBoost",
    endpoint_name=endpoint.name,
    model=model,
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

In [ ]:
ml_client.batch_deployments.begin_create_or_update(deployment)

Let's update the default deployment name in the endpoint:

In [ ]:
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint)

We can see the endpoint URL as follows:

In [ ]:
endpoint.scoring_uri

## Testing the endpoint

Once the deployment is created, it is ready to receive jobs. Let's first register a data asset so we can run the job against it. This data asset is a folder containing multiple CSV files that we want to process in parallel using the batch endpoint we just created.

In [ ]:
data_path = "heart-classifier-mlflow/heart-dataset/"
dataset_name = "heart-dataset-unlabeled"

heart_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for heart classification",
    name=dataset_name,
)

ml_client.data.create_or_update(heart_dataset_unlabeled)

In [ ]:
heart_dataset_unlabeled = ml_client.data.get(
    name="heart-dataset-unlabeled", label="latest"
)

Let's use this data as an input for the job:

In [ ]:
from azure.ai.ml import Input

input = Input(type=AssetTypes.URI_FOLDER, path=heart_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)

You can use the returned job object to check the status of the job:

In [ ]:
ml_client.jobs.get(job.name)

Notice how we are not indicating the deployment name in the invoke operation. That's because the endpoint automatically routes the job to the default deployment. Since our endpoint only has one deployment, then that one is the default one. You can target an specific deployment by indicating the argument/parameter `deployment_name`.

In [ ]:
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name, endpoint_name=endpoint.name, input=input
)

### Overwriting deployment degree of parallelism in jobs

Batch endpoints allow you to overwrite some of the parameters of the batch deployment. You can use this to control the degree of parallelization in your job depending to the size of the data input you are using:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name,
    endpoint_name=endpoint.name,
    input=input,
    params_override=[{"mini_batch_size": "20"}, {"compute.instance_count": "1"}],
)

## Exploring the results

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("named-outputs/score/predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score